In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# > Задача 1. Сколько выбросов удалять
  

С одной стороны, при удалении выбросов снижается дисперсия, что приводит к увеличению чувствительности теста. С другой стороны, при удалении выбросов уменьшается размер выборки, что приводит к уменьшению чувствительности.

Сравните мощности тестов с разной долей удаляемых данных. Используйте данные о времени работы бэкенда `2022-04-01T12_df_web_logs.csv` в период с 2022-03-01 по 2022-03-08. Уровень значимости — 0.05. Размеры групп — 1000 человек (размер выборок будет больше, так как на одного человека приходится много значений). Проверяем гипотезу о равенстве средних с помощью теста Стьюдента. Ожидаемый эффект — увеличение времени обработки на 1%. Эффект в синтетических А/В-тестах добавляем умножением на константу.

В ответ введите номера вариантов, упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между `np.quantile(values, 0.01)` и `np.quantile(values, 0.99)`. Квантили вычислять для каждой групп отдельно.

In [2]:
web_logs = pd.read_csv('2022-04-01T12_df_web_logs.csv')
web_logs.head(3)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7


In [3]:
# отсеиваем данные по датам конкретной недели
web_logs_week = web_logs[
    (web_logs['date'] >= '2022-03-01')
    & (web_logs['date'] < '2022-03-08')
]
web_logs_week.shape

(245851, 4)

In [4]:
alpha = 0.05
sample_size = 1000
eff = 0.01

# Отбираем только уникальных пользователей для нашего эксперимента.
users = web_logs_week['user_id'].unique()

In [5]:
outliers = [0.0002, 0.002, 0.02, 0.1, 0.2]
result = []

for index, quant in enumerate(outliers):
    pvalues = []

    for _ in range(1000):
        # Делаем две выборки размером 1000, которые не пересекаются между собой.
        users_a, users_b = np.random.choice(users, (2, sample_size), replace=False)
        # Отбираем необходимые данные для теста и добавляем в экспериментальную группу наш эффект.
        a = web_logs_week.query('user_id in @users_a').load_time
        b = web_logs_week.query('user_id in @users_b').load_time * (1 + eff)
        
        # Отсеиваем все что меньше нижнего квантиля и больше верхнего квантиля для наших распределений
        a_mask = (a > np.quantile(a, quant / 2)) & (a < np.quantile(a, 1 - quant / 2))
        b_mask = (b > np.quantile(b, quant / 2)) & (b < np.quantile(b, 1 - quant / 2))
        group_a = a[a_mask]
        group_b = b[b_mask]
        
        # Проводим t-test и записываем pvalue для рпоследующего рассчета мощности
        pvalue = stats.ttest_ind(group_a, group_b).pvalue
        pvalues.append(pvalue)

    # Рассчитываем мощность при заданном квантиле
    power = np.mean((np.array(pvalues) < alpha).astype(int))
    result.append([index + 1, quant, power])

    print(f'При удаленни {quant * 100}% выбросов мощность теста равна {power}')
    
result_sorted = sorted(result, key=lambda power: power[2])
answer_list = [i[0] for i in result_sorted]
print(f'''
ответ: {''.join(map(str, reversed(answer_list)))}.
''')

При удаленни 0.02% выбросов мощность теста равна 0.097
При удаленни 0.2% выбросов мощность теста равна 0.338
При удаленни 2.0% выбросов мощность теста равна 0.93
При удаленни 10.0% выбросов мощность теста равна 0.947
При удаленни 20.0% выбросов мощность теста равна 0.973

ответ: 54321.



# > Задача 2. Сколько выбросов удалять — 2
  

Выполните то же задание, изменив способ добавления эффекта. Эффект в синтетических А/В-тестах добавляем добавлением константы к 1% данных.

В ответ введите номера вариантов упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между `np.quantile(values, 0.01)` и `np.quantile(values, 0.99)`. Квантили вычислять для каждой группы отдельно.

In [7]:
alpha = 0.05
sample_size = 1000
eff = 0.01

outliers = [0.0002, 0.002, 0.02, 0.1, 0.2]
result = []

for index, quant in enumerate(outliers):
    pvalues = []

    for _ in range(1000):
        # Делаем две выборки размером 1000, которые не пересекаются между собой.
        users_a, users_b = np.random.choice(users, (2, sample_size), replace=False)
        # Отбираем необходимые данные для теста и добавляем в экспериментальную группу наш эффект.
        a = web_logs_week[web_logs_week['user_id'].isin(users_a)].load_time.values
        b = web_logs_week[web_logs_week['user_id'].isin(users_b)].load_time.values
        mean = np.mean(b)
        # Рандомно выбираем индексы для нашей выборки в 1% от всех значений
        indexes = np.random.choice(range(len(b)), int(len(b) * 0.01), False)
        # Для равноммерного распределения эффекта делим его на количество значений, к которым мы его добавляем
        b[indexes] += eff * mean * len(b) / len(indexes)
        
        # Отсеиваем все что меньше нижнего квантиля и больше верхнего квантиля для наших распределений
        a_mask = (a > np.quantile(a, quant / 2)) & (a < np.quantile(a, 1 - quant / 2))
        b_mask = (b > np.quantile(b, quant / 2)) & (b < np.quantile(b, 1 - quant / 2))
        group_a = a[a_mask]
        group_b = b[b_mask]
        
        # Проводим t-test и записываем pvalue для рпоследующего рассчета мощности
        pvalue = stats.ttest_ind(group_a, group_b).pvalue
        pvalues.append(pvalue)

    # Рассчитываем мощность при заданном квантиле
    power = np.mean((np.array(pvalues) < alpha).astype(int))
    result.append([index + 1, quant, power])

    print(f'При удаленни {quant * 100}% выбросов мощность теста равна {power}')
    
result_sorted = sorted(result, key=lambda power: power[2])
answer_list = [i[0] for i in result_sorted]
print(f'''
ответ: {''.join(map(str, reversed(answer_list)))}.
''')

При удаленни 0.02% выбросов мощность теста равна 0.092
При удаленни 0.2% выбросов мощность теста равна 0.349
При удаленни 2.0% выбросов мощность теста равна 0.437
При удаленни 10.0% выбросов мощность теста равна 0.312
При удаленни 20.0% выбросов мощность теста равна 0.331

ответ: 32541.

